In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import gc
import re

from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import scale, minmax_scale
from scipy.stats import norm

import lightgbm as lgb

import warnings
warnings.simplefilter(action='ignore')
warnings.filterwarnings('ignore')

# change to path
PATH='/content/drive/My Drive/Colab Notebooks/nba/'
os.chdir(PATH)

## Read in

In [3]:
train = pd.read_csv(f'{PATH}/dataset/training_set.csv', encoding = 'ISO-8859-1')
test = pd.read_csv(f'{PATH}/dataset/holdout_set.csv', encoding = 'ISO-8859-1')

train.head()

,Engagements,Followers at Posting,Created,Type,Description
0,502093,36984682,2019-05-21 23:30:51 EDT,Video,The @raptors bench trio of @sergeibaka @norman...
1,603380,36984682,2019-05-21 22:53:33 EDT,Video,@kyle_lowry7 pulls from deep for the @raptors ...
2,603380,36984682,2019-05-21 22:19:58 EDT,Video,@k_mid22 with some english on the @bucks dime!
3,725100,36984682,2019-05-21 22:02:41 EDT,Video,Kawhi punches it home with the left on TNT!
4,661446,36984682,2019-05-21 20:47:49 EDT,Video,@giannis_an34 goes baseline early to rock the ...


In [4]:
len(train), len(test)

(7766, 1000)

## Preprocessing

In [5]:
print('Missing descriptions:', train['Description'].isna().sum())

train['Description'].fillna('', inplace=True)

print('Missing descriptions:', train['Description'].isna().sum())

Missing descriptions: 14
Missing descriptions: 0


## Time feature

*   Hour, minute, second
*   Day of the week
*   Month
*   Quarter
*   Year
*   Number of posts on the day
*   Number of posts 2 days before (since playoffs have a one-day gap)



In [0]:
def str_to_date(s, split):
    return s.split(' ')[split]
  
def add_datepart(df, fldname, drop=True, time=False):
    """
    Taken from fast.ai
    Helper function that adds columns relevant to a date.
    """
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)
      
def get_time_split(time):
    time_minute = time.split(':')[:2]
    return int(''.join(time_minute))
  
def get_hour(time):
    return int(time.split(':')[0])
  
def get_minute(time):
    return round(int(time.split(':')[1]), -1)

In [0]:
def preprocess(df):
    df['date'] = df.apply(lambda x: str_to_date(x['Created'], 0), axis=1)
    df['time'] = df.apply(lambda x: str_to_date(x['Created'], 1), axis=1)
    df['tz'] = df.apply(lambda x: str_to_date(x['Created'], 2), axis=1)
    
    # get more features on date
    df['date'] = pd.to_datetime(df['date'])
    
    # date
    df['date_tsfm'] = pd.to_datetime(df['date'])
    
    add_datepart(df, 'date_tsfm')

    time_df = pd.get_dummies(df['Type'], 
                         prefix='Type')

    # get only time and hour
    df['time_split'] = df['time'].apply(get_time_split)
    df['hour'] = df['time'].apply(get_hour)
    df['minute'] = df['time'].apply(get_minute)   
    
    df = pd.concat([df, time_df],
                    axis=1)
    
    return df

In [0]:
train = preprocess(train)
test = preprocess(test)

To get the number of posts on the day, I first concatenated the `train` and `test` to see how many posts were posted on the day.

In [9]:
# keep track of the len of train
# so that we can split again after 
# this operation
train_len = len(train)

df = pd.concat([train, test])

df.shape

(8766, 27)

In [0]:
# get the number of posts
# for each day
date_counts = df.groupby('date')['time'].count().to_dict()

# create a new feature
# to map the number of posts 
# of the day
df['num_posts'] = df['date'].map(date_counts)

In [0]:
# get the date for 2 days ago
df['date_lag_2'] = df['date'] - np.timedelta64(2, 'D')

# create a new feature
# to map the number of posts
# 2 days ago
df['num_posts_lag_2'] = df['date_lag_2'].map(date_counts)

# replace the missing days 
# for a 2-day lag with
# the same number of posts on 
# the same day
df['num_posts_lag_2'] = df['num_posts_lag_2'].fillna(df['num_posts'])

In [12]:
# check if there is any missing values
len(df[df['num_posts_lag_2'].isna()])

0

In [0]:
# split the df
# back to train and test
train = df[:train_len].copy()
test = df[train_len:].copy()

In [14]:
train.head()

,Engagements,Followers at Posting,Created,Type,Description,date,time,tz,date_tsfmYear,date_tsfmMonth,date_tsfmWeek,date_tsfmDay,date_tsfmDayofweek,date_tsfmDayofyear,date_tsfmIs_month_end,date_tsfmIs_month_start,date_tsfmIs_quarter_end,date_tsfmIs_quarter_start,date_tsfmIs_year_end,date_tsfmIs_year_start,date_tsfmElapsed,time_split,hour,minute,Type_Album,Type_Photo,Type_Video,num_posts,date_lag_2,num_posts_lag_2
0,502093.0,36984682,2019-05-21 23:30:51 EDT,Video,The @raptors bench trio of @sergeibaka @norman...,2019-05-21,23:30:51,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2330,23,30,0,0,1,19,2019-05-19,18.0
1,603380.0,36984682,2019-05-21 22:53:33 EDT,Video,@kyle_lowry7 pulls from deep for the @raptors ...,2019-05-21,22:53:33,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2253,22,50,0,0,1,19,2019-05-19,18.0
2,603380.0,36984682,2019-05-21 22:19:58 EDT,Video,@k_mid22 with some english on the @bucks dime!,2019-05-21,22:19:58,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2219,22,20,0,0,1,19,2019-05-19,18.0
3,725100.0,36984682,2019-05-21 22:02:41 EDT,Video,Kawhi punches it home with the left on TNT!,2019-05-21,22:02:41,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2202,22,0,0,0,1,19,2019-05-19,18.0
4,661446.0,36984682,2019-05-21 20:47:49 EDT,Video,@giannis_an34 goes baseline early to rock the ...,2019-05-21,20:47:49,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2047,20,50,0,0,1,19,2019-05-19,18.0


## Text features
Try to extract the following feature for each document - `Description`.
*   Number of #
*   Number of @
*   Number of punctuations
*   Number of uppercase letters
*   Number of tokens



In [15]:
ex = train.iloc[6]['Description']
ex

'The @warriors locked in for four games to advance to the #NBAFinals presented by YouTube TV! #GatoradeZero'

In [16]:
def count_hashes(s):
    return s.count('#')
  
def count_at_signs(s):
    return s.count('@')
  
# spacy  
def count_punct(s):
    count = 0
    punct = '?!.,"$%\'()*+-/:;<=>[\\]^_`{|}~' + '“”’'    
    for p in punct: 
        if p in s: 
          count += 1    
    return count
  
def count_tokens(s):
    return len(s.split())
  
def count_upper(s):
    return len(re.findall(r'[A-Z]',s))
  
count_hashes(ex), count_at_signs(ex), count_punct(ex), count_tokens(ex), count_upper(ex)  

(2, 1, 1, 17, 11)

In [0]:
def text_features(df):
    df['num_hashes'] = df['Description'].apply(count_hashes)
    df['num_at_signs'] = df['Description'].apply(count_at_signs)
    df['num_punct'] = df['Description'].apply(count_punct)
    df['num_tokens'] = df['Description'].apply(count_tokens)
    df['num_upper'] = df['Description'].apply(count_upper)
    
text_features(train)    

In [0]:
def clean_text(x):
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x
  
train['clean_des'] = train['Description'].apply(lambda x: clean_text(x))

In [0]:
# from spacy.lang.en import English
# import spacy

# !spacy download en_core_web_md

# # spacy.load('en_core_web_md') # not working

# nlp = en_core_web_md.load()

# from spacy.tokenizer import Tokenizer
# tokenizer = Tokenizer(nlp.vocab)

# from spacy.attrs import ORTH, LEMMA

# case = [{ORTH: "@"}]

# tokenizer.add_special_case("@", case)

# # Construction from class
# from spacy.pipeline import Sentencizer
# sentencizer = Sentencizer()

# nlp.add_pipe(sentencizer)
# nlp.add_pipe(tokenizer)

In [20]:
ex = train.iloc[0]['clean_des']

ex

'The raptors bench trio of sergeibaka normanpowell4  &  fredvanvleet combine for 48 in Game 4'

In [0]:
# def extract_doc_features(text):
#     doc = nlp(text)
    
#     # original doc len
#     doc_len = len(doc)
    
#     # clean doc
#     clean_doc = clean_text(text)
    
#     # clean doc len
#     clean_doc_len = len(clean_doc.split())
    
#     print(clean_doc.split())

#     for i, token in enumerate(tokenizer(clean_doc)):
#         print(i, token)
    
#     for token in doc:
#         print(token.text, token.tag_, token.pos_, token.head, token.is_stop, token.ent_type_)

# extract_doc_features(ex)

In [22]:
gc.collect()

598

## Ideas

*   Tokenize text to get NBA team's Instagram. Perform `OneHotEncoding` on names.
*   Tokenize text to get players's Instagram. Perform `OneHotEncoding` on names. Mask names with `[TEAM]_player`.
*   Time, i.e., before or after game, playoffs or regular.
*   Understand how is `Engagement` generated.
*   Closeness of score.
*   Train FastText
https://fasttext.cc/docs/en/unsupervised-tutorial.html

In [0]:
def get_at_sign(s):
    ls =  re.findall(r"@(\w+)", s)
    return '|'.join(ls)

def get_hash(s):
    ls = re.findall(r"#(\w+)", s)
    return '|'.join(ls)

In [24]:
%%time
train['hashes'] = train['Description'].apply(get_hash)
train['at_signs'] = train['Description'].apply(get_at_sign)

CPU times: user 24.5 ms, sys: 1.07 ms, total: 25.5 ms
Wall time: 25.5 ms


For all the `hashes` and `at_signs`, remove those that appear 3 times or fewer for `hashes` and 10 times or fewer for `at_signs`. 

In [0]:
hashes_mat = train['hashes'].str.get_dummies()
at_signs_mat = train['at_signs'].str.get_dummies()

In [26]:
hashes_mat.head()

,1,100,12,15,18HoopClass,19HoopClass,2,2019,21,22,23,24,2WayPlayer,3,34,4,4MrThunder,5,50,8,82,ADayWithGrayson,ADayWithMo,ATTSlamDunk,ATTvip,AirplaneMode,AllEyesNorth,AllForOne,AmericasTeamCamp,AmexNBA,AndreIngram,AssistsOfTheWeek,BANG,BBQChickenAlert,BESTofNBA,BESTofNBA2017,BHM,BWBAfrica,BWBGlobal,BeatsStudio3Wireless,...,VerizonDunk,VetMove,VeteransDay,ViceNights,WINorGoHome,WNBA,WNBA3Point,WNBAAllStar,WNBADraft,WNBAFinals,WNBAPlayoffs,WNBAPride,WadeCounty,WarriorsAllAccess,WarriorsParade,WatchMeWork,WeBelieve,WeTheNorth,WeekofGreatness,WhateverItTakes,WhiteHot,WhyNot,WhyNotTour,WiredDifferent,WorldTrickShotDay,Year16,fannypack,feathery,globalgame,kiatipoff18,ko8e24,nba2kleague,nbabreakdown,nbacanadaseries,nbapreseason,nbarooks,themambamentality,thisiswhyweplay,wnbaallstar,wnbaplayoffs
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
at_signs_mat.head()

,1jordanbell,1ngram4,1tylerennis,1tyus,22wiggins,23,24baze,35_fitz,3dtv,3jmccollum,3tross1,44bojan,50mejri,7tlc,ATLHawks,ATT,AdidasHoops,AmericanExpress,BenSimmons,BlakeGriffin23,BreannaStewart30,BrooklynNets,Bucks,BudweiserUSA,CP3,Cavs,Celtics,DallasMavs,DallasWings,DamianLillard,DejounteMurray,DwightHoward,DwyaneWade,ESPN,FIBA,FS1,GRDrive,Gallo_Locknez,Giannis_An34,Google,...,ud40,unclejeffgreen,unclejg8,underarmour,unitedmasters,usabasketball,utahjazz,utahjazzgaming,utahjazzsl,vicoladipo,waiters3,waltdisneyworld,warriors,washmystics,washwizards,wayne_elli,wayneseldenjr,wendellcarterjr,wenyengabriel,wessywes23,wholeteamdot,willthethrillb5,willyhernangomez,winnieharlow,wisconsinherd,wnba,y0bull,yankees,ygtrece,youngamechanger,youngheirgordon,youtube,yutawatanabe12,zach_ballin30,zachcollins_33,zachlavine8,zazapachulia,zhaire_smith,zmane2,zo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
idx_keep = list(np.where(np.sum(hashes_mat.values, axis=0) > 10)[0])
hashes_keep = [hashes_mat.columns[i] for i in idx_keep]

idx_keep = list(np.where(np.sum(at_signs_mat.values, axis=0) > 30)[0])
at_signs_keep = [at_signs_mat.columns[i] for i in idx_keep]

len(hashes_keep), len(at_signs_keep)

(68, 78)

This approach reduces the number of `hashes` from 384 to 133 and the number of `at_signs` from 848 to 159. Next, I encode those words that appear less often as rare `RARE`.

In [29]:
hashes_remove = [c for c in hashes_mat.columns if c not in hashes_keep]
at_signs_remove = [c for c in at_signs_mat.columns if c not in at_signs_keep]

len(hashes_remove), len(at_signs_remove)

(316, 770)

In [0]:
# sum across all the rare words 
# for hashes and at signs
hashes_rare = hashes_mat[hashes_remove].sum(axis=1).values
at_signs_rare = at_signs_mat[at_signs_remove].sum(axis=1).values

In [0]:
hashes = hashes_mat[hashes_keep]
hashes['hashes_rare'] = hashes_rare

at_signs = at_signs_mat[at_signs_keep]
at_signs['at_signs_rare'] = at_signs_rare

In [32]:
hashes.shape, at_signs.shape

((7766, 69), (7766, 79))

In [0]:
df = pd.concat([train, hashes, at_signs],
                axis=1)

## Save the file

In [37]:
df.head()

,Engagements,Followers at Posting,Created,Type,Description,date,time,tz,date_tsfmYear,date_tsfmMonth,date_tsfmWeek,date_tsfmDay,date_tsfmDayofweek,date_tsfmDayofyear,date_tsfmIs_month_end,date_tsfmIs_month_start,date_tsfmIs_quarter_end,date_tsfmIs_quarter_start,date_tsfmIs_year_end,date_tsfmIs_year_start,date_tsfmElapsed,time_split,hour,minute,Type_Album,Type_Photo,Type_Video,num_posts,date_lag_2,num_posts_lag_2,num_hashes,num_at_signs,num_punct,num_tokens,num_upper,clean_des,hashes,at_signs,18HoopClass,ATTSlamDunk,...,kyrieirving,laclippers,lakers,lukadoncic,memgrizz,miamiheat,money23green,nbaallstar,nbahistory,nbaonespn,nbaontnt,nbasummerleague,nbatv,nuggets,nyknicks,okcthunder,orlandomagic,pacers,pelicansnba,rajonrondo,raptors,russwest44,sacramentokings,sixers,spidadmitchell,spurs,stephencurry30,suns,swish41,timberwolves,traeyoung,trailblazers,utahjazz,vicoladipo,warriors,washwizards,wnba,ygtrece,zo,at_signs_rare
0,502093.0,36984682,2019-05-21 23:30:51 EDT,Video,The @raptors bench trio of @sergeibaka @norman...,2019-05-21,23:30:51,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2330,23,30,0,0,1,19,2019-05-19,18.0,0,4,1,15,2,The raptors bench trio of sergeibaka normanpow...,,raptors|sergeibaka|normanpowell4|fredvanvleet,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
1,603380.0,36984682,2019-05-21 22:53:33 EDT,Video,@kyle_lowry7 pulls from deep for the @raptors ...,2019-05-21,22:53:33,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2253,22,50,0,0,1,19,2019-05-19,18.0,0,2,2,12,3,kylelowry7 pulls from deep for the raptors in ...,,kyle_lowry7|raptors,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,603380.0,36984682,2019-05-21 22:19:58 EDT,Video,@k_mid22 with some english on the @bucks dime!,2019-05-21,22:19:58,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2219,22,20,0,0,1,19,2019-05-19,18.0,0,2,2,8,0,kmid22 with some english on the bucks dime,,k_mid22|bucks,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,725100.0,36984682,2019-05-21 22:02:41 EDT,Video,Kawhi punches it home with the left on TNT!,2019-05-21,22:02:41,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2202,22,0,0,0,1,19,2019-05-19,18.0,0,0,1,9,4,Kawhi punches it home with the left on TNT,,,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,661446.0,36984682,2019-05-21 20:47:49 EDT,Video,@giannis_an34 goes baseline early to rock the ...,2019-05-21,20:47:49,EDT,2019,5,21,21,1,141,False,False,False,False,False,False,1558396800,2047,20,50,0,0,1,19,2019-05-19,18.0,0,1,2,10,3,giannisan34 goes baseline early to rock the ri...,,giannis_an34,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# df.to_csv(f'{PATH}/dataset/df_186.csv', index=False)

In [0]:
# codes = {
#     PHI	76ers
#     MIL	Bucks
#     CHI	Bulls
#     CLE	Cavaliers
#     BOS	Celtics
#     LAC	Clippers
#     MEM	Grizzlies
#     ATL	Hawks
#     MIA	Heat
#     CHA	Hornets
#     UTA	Jazz
#     SAC	Kings
#     NYK	Knicks
#     LAL	Lakers
#     ORL	Magic
#     DAL	Mavericks
#     BKN	Nets
#     DEN	Nuggets
#     IND	Pacers
#     NOP	Pelicans
#     DET	Pistons
#     TOR	Raptors
#     HOU	Rockets
#     SAS	Spurs
#     PHX	Suns
#     OKC	Thunder
#     MIN	Timberwolves
#     POR	Trailblazers
#     GSW	Warriors
#     WAS	Wizards
# }